In [10]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pyodbc
import pandas as pd
import os
from os.path import join
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
path='../data/raw/'

path = '../data/interim/'
sciezka_to = '../data/processed/'



In [3]:
def volumeproc(max_v, volume_from_dose):
    return 100*volume_from_dose/max_v


def get_basic(df):
    #Zwraca słownik z parametrami DVH np czy jest % lub w cGy
    basics = df.columns.str.strip()
    params = {x.split(': ')[0]:x.split(': ')[1] for x in basics}
    return params


def get_date(df):
    #Zwraca timestamp pliku DVH (ostatnia pozycja w pliku)
    data = df.iloc[-1,0]
    data = pd.to_datetime(data, format='%m/%d/%Y %I:%M:%S %p')
    return data


def get_dvh_table(df):
    #Zwraca 3-kolumnową tabelę DVH do dalszych obliczeń
    df.columns
    column_names = ['Structure Name', 'Dose', 'Volume']
    df = df.iloc[1:-1]
    df.columns = df.iloc[0].str.strip()
    df = df[1:]
    df=df[['Structure Name', 'Dose', 'Volume']]
    for i in column_names:
        df[i] = df[i].str.strip()
    for i in column_names[1:]:
        df[i] = df[i].astype(float)
    return df


def get_ptv_name(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PTV') > -1:
            find.append(str_name)
            if len(find)>1:
                for i in find:
                    if i.find('42') == -1:
                        find.remove(i)
    return find[0], len(find)


def get_piers_name(struct)->tuple:
    find=list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PIERS') > -1:
            find.append(str_name)
            
    return find[0], len(find)


def get_serce_name(struct)->tuple:
    find=list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('SERCE') > -1:
            find.append(str_name)
            
    return find[0], len(find)


def get_pluco_L_name(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PLUCO') > -1:
            find.append(str_name)
            if len(find)>1:
                for i in find:
                    
                    if i.lower().count('l') == 1:
                        find.remove(i)
    return find[0], len(find)


def get_pluco_P_name(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('PLUCO') > -1:
            find.append(str_name)
            if len(find)>1:
                for i in find:
                    if i.upper().count('P') == 1:
                        find.remove(i)
    return find[0], len(find)


def get_external_name(struct)->tuple:
    find = list()
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('EXTERNAL') > -1:
            find.append(str_name)
            if len(find)>1:
                for i in find:
                    if i.upper().count('DIBH') != 1:
                        find.remove(i)
    return find[0], len(find)


def get_lad_name(struct)->tuple:
    find=['brak']
    for str_name in struct:
        str_name_u=str_name.upper()
        if str_name_u.find('LAD') > -1:
            find[0] = str_name
            
    return find[0], len(find)


def average_dose(df, strukture_name):
    df = df.loc[df['Structure Name']==strukture_name]
    df['Shift1']=df['Volume'].shift(1)
    df['diff1'] = df['Shift1']-df['Volume']
    df['razy1'] = df['diff1']*df['Dose']

    suma_dawek = df['razy1'].sum()
    suma_vol = df['diff1'].sum()

    return round(suma_dawek/suma_vol, 3)



def get_ptv_info(df, ptv_name):
    #Info z PTV objętość, Dmax, Dmin, Dave
    df_ptv = df.loc[(df['Structure Name']==ptv_name) &
                    (df['Dose'] > 0.0)]
    max_d = df_ptv['Dose'].max()
    min_d = df_ptv['Dose'].min()
    ave_dose = average_dose(df, ptv_name)
    max_v = df_ptv['Volume'].max()

    doseP = 40.375
    volume_from_dose = get_volume_from_dose(df, ptv_name , doseP)

    volumeProc = round(volumeproc(max_v, volume_from_dose), 3)

    return max_v, max_d, min_d, ave_dose, volumeProc


def find_neighbours(value, df, colname):
    exactmatch = df[df[colname] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        lowerneighbour_ind = df[df[colname] < value][colname].idxmax()
        upperneighbour_ind = df[df[colname] > value][colname].idxmin()
        return [lowerneighbour_ind, upperneighbour_ind] 


def get_volume_from_dose(df, strukture_name, doseP):
    serched_col = 'Dose'
    df = df.loc[(df['Structure Name']==strukture_name) &
                    (df[serched_col] > 0.0)]
    
    result = find_neighbours(doseP, df, serched_col)

    if len(result)==1:
        ret = df.loc[result]['Volume'].tolist()
        return ret[0]
    else:
        #y = y1 + (x - x1) ⨯ (y2 - y1) / (x2 - x1)  

        y_val = df.loc[result]['Volume']
        x_val = df.loc[result]['Dose']

        x= doseP

        y_val = y_val.tolist()
        y1 = y_val[0]
        y2 = y_val[1]

        x_val = x_val.tolist()
        x1 = x_val[0]
        x2 = x_val[1]

        return  y1 + (x - x1) * (y2 - y1) / (x2 - x1) 
    
def get_pluco_L_info(df, struct_name):
    df = df.loc[(df['Structure Name']==struct_name) &
                    (df['Dose'] > 0.0)]
    max_v = df['Volume'].max()

    dose1 = 18
    volume_from_dose1 = get_volume_from_dose(df, struct_name , dose1)
    volumeProc1 = round(volumeproc(max_v, volume_from_dose1), 3)

    dose2 = 5
    volume_from_dose2 = get_volume_from_dose(df, struct_name , dose2)
    volumeProc2 = round(volumeproc(max_v, volume_from_dose2), 3)

    ave_dose = average_dose(df, struct_name)

    return volumeProc1, volumeProc2, ave_dose


def get_serce_info(df, struct_name):
    df = df.loc[(df['Structure Name']==struct_name) &
                    (df['Dose'] > 0.0)]
    max_v = df['Volume'].max()

    dose1 = 25
    volume_from_dose1 = get_volume_from_dose(df, struct_name , dose1)
    volumeProc1 = round(volumeproc(max_v, volume_from_dose1), 3)

    ave_dose = average_dose(df, struct_name)

    return volumeProc1, ave_dose


def get_external_info(df, struct_name):
    df = df.loc[(df['Structure Name']==struct_name) &
                    (df['Dose'] > 0.0)]
    max_v = df['Volume'].max()

    dose1 = 5
    volume_from_dose1 = get_volume_from_dose(df, struct_name , dose1)
    volumeProc1 = round(volumeproc(max_v, volume_from_dose1), 3)

    return volumeProc1


def get_piers_info(df, struct_name):
    df = df.loc[(df['Structure Name']==struct_name) &
                    (df['Dose'] > 0.0)]
    max_v = df['Volume'].max()

    dose1 = 5
    volume_from_dose1 = get_volume_from_dose(df, struct_name , dose1)
    volumeProc1 = round(volumeproc(max_v, volume_from_dose1), 3)

    ave_dose = average_dose(df, struct_name)

    return volumeProc1, ave_dose


def get_pluco_P_info(df, struct_name):
    df = df.loc[(df['Structure Name']==struct_name) &
                    (df['Dose'] > 0.0)]
    max_v = df['Volume'].max()

    dose1 = 5
    volume_from_dose1 = get_volume_from_dose(df, struct_name , dose1)
    volumeProc1 = round(volumeproc(max_v, volume_from_dose1), 3)

    ave_dose = average_dose(df, struct_name)

    return  volumeProc1, ave_dose


def get_lad_info(df, struct_name):
    try:
        df = df.loc[(df['Structure Name']==struct_name) &
                        (df['Dose'] > 0.0)]
        max_v = df['Volume'].max()

        dose1 = 25
        volume_from_dose1 = get_volume_from_dose(df, struct_name , dose1)
        volumeProc1 = round(volumeproc(max_v, volume_from_dose1), 3)

        ave_dose = average_dose(df, struct_name)
        return  volumeProc1, ave_dose
    
    except ValueError:
        return ['brak', 'brak']
    

    

## Tu tworzysz funkcje

### Tu zakończyłeś. Uzupełnij parser, tak, żeby tworzył listę obliczonych elementów

- Masz już nazwy struktur
- teraz stwórz funkje get info do struktur
- co się stanie gdy pac ma jedno płuco, albo brak piersi?
- Trzeba zrobić nowy notatnik filtrujący pliki nie spełniające kryteriów Gy i ccm

In [4]:
def parse_dvh(biger_df):
    
    df = get_dvh_table(biger_df)
    #Uzyskaj nazwy struktur
    struct = df['Structure Name'].unique()

    ptv_name = get_ptv_name(struct)
    piers_name = get_piers_name(struct)
    serce_name = get_serce_name(struct)
    pluco_L_name = get_pluco_L_name(struct)
    pluco_P_name = get_pluco_P_name(struct)
    external_name = get_external_name(struct)
    lad_name = get_lad_name(struct)



    ptv_info = get_ptv_info(df, ptv_name[0])
    pluco_L_info = get_pluco_L_info(df, pluco_L_name[0])
    serce_info = get_serce_info(df, serce_name[0])
    external_info = get_external_info(df, external_name[0])
    piers_info = get_piers_info(df, piers_name[0])
    pluco_P_info = get_pluco_P_info(df, pluco_P_name[0])
    lad_info = get_lad_info(df, lad_name[0])


    data_for_record = [*ptv_info, *pluco_L_info, *serce_info, external_info, *piers_info, *pluco_P_info, *lad_info]
    return data_for_record


In [5]:
def csv_to_record(filename):
    df = pd.read_csv(filename
                 , encoding='cp1252'
                 , delimiter='\t'
                 )
    
    basics = get_basic(df)
    dttm = get_date(df)
    dvh_data = parse_dvh(df)
    
    

    record = (basics['Patient ID'], basics['Plan Name'], *dvh_data)
    return list(record)

records = list()
for entry in os.scandir(path):
    record = csv_to_record(join(path,entry.name))
    records.append(record)
    print(record)




['68092300947', 'DIBHmotylPiersLFFF', 463.562, 47.2, 35.615, 43.555, 98.287, 16.835, 38.239, 8.038, 0.03, 1.63, 18.082, 9.497, 1.848, 0.109, 0.603, 'brak', 'brak']


## Dodawanie danych z SQL

In [13]:
sql = """
select pat.Last_Name, pat.First_Name, pat.Pat_ID1, id.ida, id.idc
FROM Patient pat
JOIN ident id ON id.pat_id1 = pat.pat_id1
where id.IDA in (\'{}\')
OR id.IDC in (\'{}\')
"""

def read_credentials_file(file_path):
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(':')
            credentials[key.strip()] = value.strip()
    return credentials


def get_sql_list(sql_str, uid, pwd, server, db):
    #Uzyskuje obiekt cursor przez połączenie z serwerem SQL

    conn = pyodbc.connect(
    #Trusted_Connection = 'Yes',
    DRIVER='{SQL Server}',
    Server=server, 
    Database=db,
    UID=uid,
    pwd=pwd
    )
    cursor = conn.cursor()
    fetch = cursor.execute(sql_str).fetchall()

    return fetch


def dict_combine(pat_names_from_sql:list, records_dict)->list:
    result=list()
    for name in pat_names_from_sql:
        line = [name[0], name[1], name[2], *records_dict[name[4]]]
        result.append(line)
        
    return result


file_path = '../../Credentials/Credentials_mosaiq.txt'

credentials_dict = read_credentials_file(file_path)


password = credentials_dict['password']
user = credentials_dict['login']
db_name = credentials_dict['databasename']
server = credentials_dict['server']
    

records_dict = {i[0]:i for i in records}


sql_where_clause = "', '".join(records_dict.keys())
sql_uni = sql.format(sql_where_clause, sql_where_clause)

pat_names = get_sql_list(sql_uni, user, password, server, db_name)

lines = dict_combine(pat_names, records_dict)


{'password': 'mosaiq', 'login': 'ReportsUser', 'databasename': 'mosaiq', 'server': 'MOSAIQ3SQL\\MOSAIQ3SQL'}


NameError: name 'records' is not defined

## Zapisywanie do Excela

In [ ]:


df1 = pd.DataFrame(lines,
                   #index=['row 1', 'row 2'],
                   columns=['Nazwisko', 'Imię', 'Id_mosaiq', 
                    'PESEL',
                    'Nazwa_planu',
                    'V PTV ccm',
                    'Dmax',
                    'Dmin',
                    'Dsrednia',
                    'V95%dawki (pokrycie)%',
                    'V18Pluco %',
                    'V5PlucoL%',
                    'DsrplucoL',
                    'V25 serce %',
                    'Dsr Serce',
                    'V5 external %',
                    'Dsr Piers P',
                    'V5 piers P%',
                    'V5 pluco P%',
                    'Dsr Pluco P Gy',
                    'LAD V25 ',
                    'LAD Dśred'])


df1.head()

# Sprawdzanie Daty planu

In [ ]:
conn = pyodbc.connect(
    #Trusted_Connection = 'Yes',
    DRIVER='{SQL Server}',
    Server='MOSAIQ3SQL\MOSAIQ3SQL',
    Database='mosaiq',
    UID=user,
    pwd=password
    )

query_base = """
select pat.Birth_DtTm DOB, rtp.Create_DtTm Dadanie_planu, rtp.Name Nazwa_planu, id.idc PESEL
FROM Patient pat
JOIN ident id ON id.pat_id1 = pat.pat_id1
JOIN RtPlan rtp ON rtp.pat_id1 = pat.pat_id1
where 
id.IDC in (\'{}\')
"""

sql_plan_dt = query_base.format(sql_where_clause)

# Query Database and Create DataFrame Using Results
df_sql = pd.read_sql_query(sql_plan_dt, conn)
df_sql.info()

In [ ]:
merged_df = pd.merge(df1, df_sql, on=['PESEL', 'Nazwa_planu'], how='outer')


In [ ]:
if df1.shape[0] == merged_df.shape[0]:
    df1=merged_df


wb = openpyxl.Workbook()
ws = wb.active
 
for i in dataframe_to_rows(df1, index=False, header=True):
    ws.append(i)  # zapisujemy dane
 
wb.save('../data/processed/StatystykaDIBH_output.xlsx')